In [1]:
import tensorflow as tf

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [5]:
x = tf.placeholder(tf.float32, [None, 28, 28, 1])
#cnn은 이미지 자체로 인풋을 주고 그걸로 학습하므로 28*28, *1은 흑백 표현 위한 것. 그래서 *3(rgb)가 아니라 *1channel
y = tf.placeholder(tf.float32, [None, 10])

In [6]:
#3*3=kernel size, input channel=1, output=32
#stride= 가로 세로로 한칸씩 뛰어넘는다. 맨처음이랑 마지막은 보통 1로 고정, 두번째 세번째가 가로세로 픽셀을 의미. 얼마나 움직일 것인가를 알려준다.
W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev=0.01))
L1 = tf.nn.conv2d(x, W1, strides =[1, 1, 1, 1], padding='SAME')

In [7]:
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [8]:
W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev=0.01))
L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [9]:
W3 = tf.Variable(tf.random_normal([7*7*64, 256], stddev=0.01))
L3 = tf.reshape(L2, [-1, 7*7*64])
L3 = tf.matmul(L3, W3)
L3 = tf.nn.relu(L3)

In [10]:
W4 = tf.Variable(tf.random_normal([256, 10], stddev=0.01))
model = tf.matmul(L3, W4)

In [12]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

In [13]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [15]:
batch_size=100
total_batch = int(mnist.train.num_examples / batch_size)

In [16]:
for epoch in range(10):
    total_cost = 0
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape(-1, 28, 28, 1)
        _, cost_val = sess.run([optimizer, cost], feed_dict={x:batch_xs, y:batch_ys})
        total_cost += cost_val
    print('Epoch:', '%04d'%(epoch+1), 'Avg.cost = ', '{:,.3f}'.format(total_cost/total_batch))

Epoch: 0001 Avg.cost =  0.313
Epoch: 0002 Avg.cost =  0.085
Epoch: 0003 Avg.cost =  0.060
Epoch: 0004 Avg.cost =  0.043
Epoch: 0005 Avg.cost =  0.035
Epoch: 0006 Avg.cost =  0.028
Epoch: 0007 Avg.cost =  0.022
Epoch: 0008 Avg.cost =  0.020
Epoch: 0009 Avg.cost =  0.016
Epoch: 0010 Avg.cost =  0.015


In [17]:
correctness = tf.equal(tf.argmax(model, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correctness, tf.float32))
print('Accuracy', sess.run(accuracy, feed_dict={x:mnist.test.images.reshape(-1, 28, 28, 1), y:mnist.test.labels}))

Accuracy 0.9869


## dropout

In [18]:
dropout_prob = tf.placeholder(tf.float32)

In [19]:
L3 = tf.nn.dropout(L3, dropout_prob)

In [ ]:
for epoch in range(10):
    total_cost = 0
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape(-1, 28, 28, 1)
        _, curr_loss = sess.run([optimizer, cost], feed_dict={x:batch_xs, y:batch_ys, dropout_prob:0.7})
        total_cost += curr_loss
    print('Epoch:', '%04d'%(epoch+1), 'Avg.cost = ', '{:,.3f}'.format(total_cost/total_batch))

In [ ]:
correctness = tf.equal(tf.argmax(model, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correctness, tf.float32))
print('Accuracy', sess.run(accuracy, feed_dict={x:mnist.test.images.reshape(-1, 28, 28, 1), y:mnist.test.labels, dropout_prob:1}))

In [ ]:
W4 = tf.Variable(tf.random_normal([4*4*128, 256], stddev=0.01))
L4 = tf.reshape(L3, [-1, 4*4*128])
L4 = tf.matmul(L4, W4)
L4 = tf.nn.relu(L4)
L4 = tf.nn.dropout(L4, dropout_prob)